The cell below gives the format of sent data depending on the spi command.

In [ ]:
cmd_format = {
    "//": "//%s",                                       # Display at the output
    "MEASURE_FREQ_VCO": "MEASURE_FREQ_VCO %d %0d %0d",  # MEASURE_FREQ_VCO CLKs MINF MAXF
    "ENABLE_AMS": "ENABLE_AMS",
    "DISABLE_AMS": "DISABLE_AMS",
    "WAIT_NS": "WAIT_NS %d",                            # WAIT_NS time_in_ns
    "WTSV": "WTSV %s %x",                               # WAIT for signal value
    "BEHR": "BEHR %x",                                  # Bist error halt routine N times
    "PSV": "PSV %s %x",                                 # PUT signal value (IO on DUT)
    "REGWR": "REGWR %x %x %x",                          # REGWR ADDR16 MASK32 DATA32
    "REGRD": "REGRD %x",                                # REGRD ADDR16
    "MEMWR": "MEMWR %x %x %x %x %x %x %x %x %x",        # MEMWR ADDR16 DATA32_31 ... DATA32_0
    "MEMRD": "MEMRD %x",                                # MEMRD ADDR16
    "MEMWRM": "MEMWRM %x %x %x %x %x %x %x %x",         # MEMWRM DATA32_31 ... DATA32_0
    "MEMRDM": "MEMRDM",                                 # MEMRDM
    "REGCMP": "REGCMP %x %x",                           # REGCMP MASK32 CMPDATA32
    "MEMCMP": "MEMCMP %x %x %x %x %x %x %x %x"          # MEMCMP CMPDATA32_7 ... CMPDATA32_0
}

In [ ]:
import re

"""

This function parses the test file into valid command, addr and data 
to transmit to the SPI in order to do the tests.

Add the command to parsed_values

"""

def run_parse_loop(test_file, result_file_html, result_file_txt) -> list[list[int]]: 

    parsed_file = [] # list of all parsed lines from the test file 
    parsed_line = [] # one parsed line from the test file : command, addr, bytes 

    # Open and read the file
    for line in test_file:
        line = line.strip()  # Remove extra spaces and newline characters
        if line.startswith("//"): # Ignore comments 
            continue
        parts = line.split() # Split the line into a list of string based on spaces
        # if not (parts) :
        #     empty_line(result_file) # function to create if want to uncomment this 
        if (parts) :
            command = parts[0]
            if command in cmd_format :
                parsed_line = extract_data_from_line(result_file_html, result_file_txt, command, line)      
            else:
                unknown_cmd(result_file_html, result_file_txt, command)
            parsed_file.append(parsed_line)
    return parsed_file



"""

This function extracts ADDR and Bytes data from one line of the test file given a certain command

"""
def extract_data_from_line(result_file_html, result_file_txt, command:str, line:str)-> list[str]:
    parsed_line = [command]
    format = cmd_format[command]
    pattern = format.replace("%d", r"(\d+)") # Match with decimal values in test file 
    pattern = pattern.replace("%x", r"([0-9a-fA-F]+)") # Match with hexadecimal values in test file 
    match = re.match(pattern, line) 
    if match:
        for data in match.groups():
            parsed_line.append(data) 
        # print(f"Command: {command}, Parsed Values: {parsed_values}")
    else:
        error_parsing_line(result_file_html, result_file_txt, line)
    return parsed_line